In [4]:
from tensorflow import keras

In [2]:
import numpy as np

In [5]:
path = keras.utils.get_file('neitzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

614400/600901 [==============================] - 2s 3us/step


In [7]:
text = open(path).read().lower()

In [8]:
print('Corpus Length' , len(text))

Corpus Length 600893


In [9]:
maxlen = 60

In [10]:
step = 3

In [16]:
sentences = []

In [18]:
next_chars = []

In [20]:
for i in range(0 , len(text) - maxlen , step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 200278


In [22]:
sentences[0]

'preface\n\n\nsupposing that truth is a woman--what then? is the'

In [23]:
chars = sorted(list(set(text)))

In [24]:
print('Unique characters:' , len(chars))

Unique characters: 57


In [27]:
char_indices = dict((char , chars.index(char)) for char in chars)

In [28]:
print('Vectorization')

Vectorization


In [30]:
x = np.zeros((len(sentences) , maxlen , len(chars)) , dtype=bool)

In [31]:
y = np.zeros((len(sentences) , len(chars) ) , dtype=bool)

In [33]:
x.shape

(200278, 60, 57)

In [35]:
for i , sentence in enumerate(sentences):
    for t , char in enumerate(sentence):
        x[i , t , char_indices[char]] = 1
    y[i , char_indices[next_chars[i]]] = 1

In [36]:
from keras import layers

In [37]:
model = keras.models.Sequential()

In [38]:
model.add(layers.LSTM(128 , input_shape=(maxlen , len(chars))))

In [39]:
model.add(layers.Dense(len(chars) , activation='softmax'))

In [43]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.001)

In [44]:
model.compile(loss='categorical_crossentropy' , optimizer=optimizer)

In [45]:
def sample(preds , temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1 , preds , 1)
    return np.argmax(probas)

In [48]:
import random
import sys

for epoch in range(1,60):
    print('epoch' , epoch)
    model.fit(x ,y , batch_size = 128, epochs = 1)
    start_index = random.randint(0 , len(text)  - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [ 0.2 , 0.5 , 1.0 , 1.2 ]:
        print('-- temperature:' , temperature)
        print(generated_text)

epoch 1


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.